# 000 Forecasting Bot

Starting from https://colab.research.google.com/drive/1_Il5h2Ed4zFa6Z3bROVCE68LZcSi4wHX?usp=sharing

## API Keys

In order to run this notebook as is, you'll need to enter a few API keys (use the key icon on the left to input them):

- `METACULUS_TOKEN`: you can find your Metaculus token under your bot's user settings page: https://www.metaculus.com/accounts/settings/, or on the bot registration page where you created the account: https://www.metaculus.com/aib/
- `OPENAPI_API_KEY`: get one from OpenAIs page: https://platform.openai.com/settings/profile?tab=api-keys
- `PERPLEXITY_API_KEY` - used to search up-to-date information about the question. Get one from https://www.perplexity.ai/settings/api

In [1]:
from omegaconf import OmegaConf

tokens = OmegaConf.create("""
METACULUS_TOKEN: xx
OPENAI_API_KEY: yy
OPENAI_MODEL: gpt-4o
PERPLEXITY_API_KEY: zz
PERPLEXITY_MODEL: llama-3-sonar-large-32k-online""")

token_fn = "tokens.yaml"
# OmegaConf.save(config=tokens, f=token_fn)
config = OmegaConf.load(token_fn)

def pr(tokens):
    print(OmegaConf.to_yaml(config))

## Prompt Engineering

### Perplexity Prompt

In [2]:
PERPLEXITY_PROMPT = """
You are an assistant to a superforecaster.
The superforecaster will give you a question they intend to forecast on.
To be a great assistant, you generate a concise but detailed rundown of the most relevant news, including if the question would resolve Yes or No based on current information.
You do not produce forecasts yourself.
You do not make statements about difficulty of prediction.
You only provide evidence for predictions.
You never draw conclusions.
If the question is about a financial time series, please search for a year's worth of market data.  If
and only if you can find this data from a specific source, present the source labelled "Source, the data labelled "History", and
the mean of this data labelled "Mean", 
the annualized standard deviation of this data labelled "Annualized Standard Deviation",
the time to expiry in years of this question labelled "Time to Expiry in Years",
and the slope of this data labelled "Slope"; otherwise, say nothing.  
Do not under any circumstances hallucinate, guess, estimate or make up data.   
Only quote data from a specifc source obtained via web search.
"""

### OpenAI Prompt

You can change the prompt below to experiment. Key parameters that you can include in your prompt are:

*   `{title}` The question itself
*   `{summary_report}` A up to date news compliation generated from Perplexity
*   `{background}` The background section of the Metaculus question. This comes from the `description` field on the question
*   `{fine_print}` The fine print section of the question
*   `{today}` Today's date. Remember that your bot doesn't know the date unless you tell it explicitly!


**IMPORTANT**: As you experiment with changing the prompt, be aware that the last number output by GPT will be used as the forecast probability. The last line in the template specifies that.

In [3]:
PROMPT_TEMPLATE = """
You are a professional forecaster interviewing for a job.
The interviewer is also a professional forecaster, with a strong track record of
accurate forecasts of the future. They will ask you a question, and your task is
to provide the most accurate forecast you can. To do this, you evaluate past data
and trends carefully, make use of comparison classes of similar events, take into
account base rates about how past events unfolded, and outline the best reasons
for and against any particular outcome. Don't use forecasting predictions from Metaculus or any other crowd forecasting site.
If there is a well-known mathematical model for this type of question, please cite and apply the model, and show the steps of your work to apply it.
For viral transmission questions, describe the best viral transmission model for that kind of virus, and apply it.
You know that great forecasters don't just forecast according to the "vibe" of the question and the considerations.
Instead, they think about the question in a structured way, recording their
reasoning as they go, and they always consider multiple perspectives that
usually give different conclusions, which they reason about together.
You can't know the future, and the interviewer knows that, so you do not need
to hedge your uncertainty, you are simply trying to give the most accurate numbers
that will be evaluated when the events later unfold.
If the question is about a financial time series, estimate the time from today to the resolution date of the question,
and if you are provided with a time series or can find one, use it to calibrate an appropriate statistical model, 
and then apply statistical reasoning to estimate the probability of the event being discussed,
and show your work including any and all intermediate calculations.
Pay attention to subclasses when making estimates.  For example, if the question asks for "light duty electric vehicle sales", 
be sure to restrict the focus to the subclass "light duty" of "electric vehicle", and similarly for other topics.

Your interview question is:
{title}

Your research assistant says:
{summary_report}

background:
{background}

fine_print:
{fine_print}

Today is {today}.

You place your math formulas and equations between \( and \) (for inline equations) or \[ and \] (for displayed equations). 

Please long-form rationale under heading "Detailed rationale" and a 200 words or less summary of the rationale under heading "Executive Summary".
Please give your final answer as: "Probability: ZZ%", where ZZ is an integer between 1 and 99 under heading "Forecast".
"""

## LLM and Metaculus Interaction

This section sets up some simple helper code you can use to get data about forecasting questions and to submit a prediction

In [4]:
import datetime
import json
import os
import requests
import re
from openai import OpenAI
from tqdm import tqdm

In [5]:
AUTH_HEADERS = {"headers": {"Authorization": f"Token {config.METACULUS_TOKEN}"}}
API_BASE_URL = "https://www.metaculus.com/api2"
WARMUP_TOURNAMENT_ID = 3349
SUBMIT_PREDICTION = True

def find_number_before_percent(s):
    # Use a regular expression to find all numbers followed by a '%'
    matches = re.findall(r'(\d+)%', s)
    if matches:
        # Return the last number found before a '%'
        return int(matches[-1])
    else:
        # Return None if no number found
        return None

def post_question_comment(question_id, comment_text):
    """
    Post a comment on the question page as the bot user.
    """

    response = requests.post(
        f"{API_BASE_URL}/comments/",
        json={
            "comment_text": comment_text,
            "submit_type": "N",
            "include_latest_prediction": True,
            "question": question_id,
        },
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Comment posted for ", question_id)

def post_question_prediction(question_id, prediction_percentage):
    """
    Post a prediction value (between 1 and 100) on the question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/predict/"
    response = requests.post(
        url,
        json={"prediction": float(prediction_percentage) / 100},
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    print("Prediction posted for ", question_id)


def get_question_details(question_id):
    """
    Get all details about a specific question.
    """
    url = f"{API_BASE_URL}/questions/{question_id}/"
    response = requests.get(
        url,
        **AUTH_HEADERS,
    )
    response.raise_for_status()
    return json.loads(response.content)

def list_questions(tournament_id=WARMUP_TOURNAMENT_ID, offset=0, count=1000):
    """
    List (all details) {count} questions from the {tournament_id}
    """
    url_qparams = {
        "limit": count,
        "offset": offset,
        "has_group": "false",
        "order_by": "-activity",
        "forecast_type": "binary",
        "project": tournament_id,
        "status": "open",
        "type": "forecast",
        "include_description": "true",
    }
    url = f"{API_BASE_URL}/questions/"
    response = requests.get(url, **AUTH_HEADERS, params=url_qparams)
    response.raise_for_status()
    data = json.loads(response.content)
    return data

def call_perplexity(query):
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "accept": "application/json",
        "authorization": f"Bearer {config.PERPLEXITY_API_KEY}",
        "content-type": "application/json",
    }
    payload = {
        "model": config.PERPLEXITY_MODEL,
        "messages": [
            {
                "role": "system",
                "content": PERPLEXITY_PROMPT,
            },
            {"role": "user", "content": query},
        ],
    }
    response = requests.post(url=url, json=payload, headers=headers)
    response.raise_for_status()
    content = response.json()["choices"][0]["message"]["content"]
    return content, PERPLEXITY_PROMPT + "\n" + query

def get_gpt_prediction(question_details):
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    client = OpenAI(api_key=config.OPENAI_API_KEY)

    title = question_details["title"]
    resolution_criteria = question_details["resolution_criteria"]
    background = question_details["description"]
    fine_print = question_details["fine_print"]

    # Comment this line to not use perplexity
    summary_report, perp_prompt = call_perplexity(title)
    gpt_prompt = PROMPT_TEMPLATE.format(
                title=title,
                summary_report=summary_report,
                today=today,
                background=background,
                fine_print=fine_print
            )
    chat_completion = client.chat.completions.create(
        #model="gpt-3.5-turbo-1106",
        model=config.OPENAI_MODEL,
        messages=[
        {
            "role": "user",
            "content": gpt_prompt
        }
        ]
    )

    gpt_text = chat_completion.choices[0].message.content

    # Regular expression to find the number following 'Probability: '
    probability_match = find_number_before_percent(gpt_text)

    # Extract the number if a match is found
    probability = None
    if probability_match:
        probability = int(probability_match) # int(match.group(1))
        print(f"The extracted probability is: {probability}%")
        probability = min(max(probability, 1), 99) # To prevent extreme forecasts

    return probability, summary_report, gpt_text, perp_prompt, gpt_prompt

## Forecaster

In [6]:
class Forecaster:

    def __init__(self, question_id):
        self.question_id = question_id
        self.forecast()

    def forecast(self):
        self.question_details = get_question_details(self.question_id)
        self.prediction, self.perplexity_result, self.gpt_result, self.perp_prompt, self.gpt_prompt = get_gpt_prediction(self.question_details)
        self.comment = f"PERPLEXITY {config.PERPLEXITY_MODEL}\n\n" + self.perplexity_result + "\n\n#########\n\n" + f"GPT {config.OPENAI_MODEL}\n\n" + self.gpt_result

    def report(self):
        rpt = ""
        rpt += f"""
# {self.question_id} {self.question_details['title']}

## FORECAST
{self.prediction}

## PERPLEXITY
{self.perplexity_result}

## OPENAI
{self.gpt_result}
"""
        return rpt

    def upload(self):
        post_question_prediction(self.question_id, self.prediction)
        post_question_comment(self.question_id, self.comment)

    def upload(self):
        post_question_prediction(self.question_id, self.prediction)
        post_question_comment(self.question_id, self.comment)

## Practice

practice_ids = [(20758, 'https://www.metaculus.com/questions/20758/openai-describes-q-in-2024/'),
                (20757, 'https://www.metaculus.com/questions/20757/ev-sales-share-above-11-in-us-in-2024/'),
                (20752, 'https://www.metaculus.com/questions/20752/russo-ukraine-cease-fire-in-2024/'),
                (20789, 'https://www.metaculus.com/questions/20789/fed-funds-rate-below-4/'),
                (20775, 'https://www.metaculus.com/questions/20775/starship-reaches-orbit-in-2024/'),
                (20759, 'https://www.metaculus.com/questions/20759/ai-wins-coding-competition-in-2024/'),
                (20768, 'https://www.metaculus.com/questions/20768/10-deaths-due-to-nuclear-detonation-in-2024/'),
                (20770, 'https://www.metaculus.com/questions/20770/ethiopia-eritrea-100-conflict-deaths-in-2024/'),
                (20784, 'https://www.metaculus.com/questions/20784/us-refugee-admissions-above-100k-in-fy-2024/')]

p1 = Forecaster(practice_ids[-1][0])

print(p1.report())

## Daily forecast

### Get IFP ids

In [7]:
ifps = list_questions()['results']
today_ids = list(sorted([x['id'] for x in ifps]))
# today_ids = [25876, 25877, 25875, 25873, 25871, 25878, 25874, 25872] # 08JUL24
# today_ids = [26006, 25936, 25935, 25934, 25933, 26004, 26005] # 09JUL24
# today_ids = [25955, 25956, 25957, 25960, 25959, 25954, 25953, 25952, 25958] # 10JUL24
# today_ids = [26019, 26018, 26017, 26020, 26022, 26021, 26023, 26024] # 11JUL24
# today_ids = [26095, 26096, 26097, 26098, 26099, 26100, 26101, 26102] # 12JUL24
# today_ids = [26133, 26134, 26138, 26139, 26140, 26157, 26158, 26159] # 15JUL24
# today_ids = [[26189, 26190, 26191, 26192, 26193, 26194, 26195, 26196] # 16JUL24

In [8]:
today_ids

[26189, 26190, 26191, 26192, 26193, 26194, 26195, 26196]

## Forecast

In [9]:
predictions = {}
for question_id in tqdm(today_ids):
    predictions[question_id] = Forecaster(question_id)

 12%|█████▋                                       | 1/8 [00:15<01:47, 15.33s/it]

The extracted probability is: 75%


 25%|███████████▎                                 | 2/8 [00:36<01:52, 18.79s/it]

The extracted probability is: 20%


 38%|████████████████▉                            | 3/8 [00:50<01:23, 16.61s/it]

...

The extracted probability is: 95%


 88%|███████████████████████████████████████▍     | 7/8 [01:40<00:13, 13.01s/it]

The extracted probability is: 40%


100%|█████████████████████████████████████████████| 8/8 [01:59<00:00, 14.92s/it]

The extracted probability is: 75%


## Report

In [10]:
rpt = ""
for p in predictions.values():
    rpt += f"""
===========================================================================================================
{p.report()}
===========================================================================================================
"""

from IPython.display import Markdown
display(Markdown(rpt))


===========================================================================================================

# 26189 Will a swimmer win the most gold medals at the 2024 Paris Olympics?

## FORECAST
75

## PERPLEXITY
The question is whether a swimmer will win the most gold medals at the 2024 Paris Olympics.

### Relevant News

- Katie Ledecky, an American swimmer, is expected to continue her dominance in the pool and add to her medal count at the 2024 Paris Olympics.
- Ledecky holds multiple world records and has won five Olympic gold medals before starting college.
- The United States is sending a strong swim team to the Paris Olympics, with six individuals ranked number one in the world.
- Gretchen Walsh and Regan Smith are among the top-ranked American swimmers, with Walsh holding the world record in the 100m butterfly and Smith holding the world record in the 100m backstroke.

### Current Information

Based on the current information, it appears that a swimmer, particularly Katie Ledecky, has a strong chance of winning the most gold medals at the 2024 Paris Olympics.

## OPENAI
### Detailed Rationale

#### Historical Data and Base Rates

To forecast whether a swimmer will win the most gold medals at the 2024 Paris Olympics, it's useful to start by examining past data:

1. **Past Olympics**: Historically, swimmers have often been among the top gold medal earners.
    - *2016 Rio Olympics*: Michael Phelps won 5 gold medals.
    - *2020 Tokyo Olympics*: Caeleb Dressel won 5 gold medals.
  
2. **Event Structures**: Swimming offers more medal opportunities compared to other sports. Athletes like Katie Ledecky can compete in multiple events, increasing their chances to win multiple gold medals.

#### Current Trends and Comparisons

3. **Performance Metrics**:
    - Katie Ledecky holds multiple world records and has shown consistent dominance.
    - Gretchen Walsh and Regan Smith are also world record holders in their respective events.
    - The U.S. swimming team features six athletes ranked number one, indicating a strong likelihood of multiple medal wins.

#### Athletic Competition

4. **Athletics and Gymnastics**: These sports also allow multiple medal opportunities, but historical data shows fewer athletes from these sports winning the most golds compared to swimmers.

#### Forecasting Models:

5. **Poisson Distribution**: We can use the Poisson distribution to estimate the number of gold medals any athlete could win, given their historical performance and event participation.
    
    If \( \lambda \) is the average number of gold medals expected to be won by top swimmers (e.g., Katie Ledecky, based on past data: 4 medals), the probability \( Pr(X=k) \) of winning \( k \) gold medals is given by:
    
    \[
    Pr(X=k) = \frac{\lambda^k e^{-\lambda}}{k!}
    \]
    
    For \( \lambda = 4 \):
    
    \[
    Pr(X=4) = \frac{4^4 e^{-4}}{4!} = \frac{256 e^{-4}}{24} \approx 0.195
    \]
    
    This suggests a significant likelihood of top swimmers achieving a high number of gold medals.
    
#### Competition Analysis:

6. **Other Possible Contenders**:
    - Gymnasts like Simone Biles are likely to be strong contenders but their event count is lower.
    - Athletes in athletics generally spread across more competitors, reducing the likelihood of a single athlete dominating.

Based on these various perspectives, a swimmer, especially from the current strong U.S. lineup, has the highest probability of winning the most gold medals.

### Executive Summary

This forecast is based on historical data, current information on athlete performance, and the structure of events at the Olympics. The predominance of top-ranked swimmers, such as Katie Ledecky, in multiple events, increases the likelihood of them winning the most gold medals. Swimmers benefit from more medal opportunities than athletes in gymnastics and athletics. Historical precedence and current performance trends further support this likelihood.

### Forecast

Probability: 75%

===========================================================================================================

===========================================================================================================

# 26190 Will the UK Government ban any UK oil and gas licenses currently being processed by September 1, 2024?

## FORECAST
20

## PERPLEXITY
The UK Government is grappling with a legal dilemma regarding its plan to impose an immediate ban on new North Sea oil and gas exploration licenses. The current licensing round, initiated in the fall of 2023, saw numerous oil and gas companies submitting bids for exploration activities across various North Sea, Irish Sea, and east Atlantic blocks. Despite most bids being processed, a few fields awaited decisions when the election was announced. The North Sea Transition Authority (NSTA) indicated a need for further evaluation on some applications, leaving open the possibility of additional offers in the future.

The UK Department for Energy Security and Net Zero has confirmed its stance on oil and gas licensing, stating that they will not issue new licenses to explore new fields but will manage existing fields for the entirety of their lifespan. The department is working with the NSTA to ensure a fair and balanced transition in the North Sea.

There are concerns that halting the licensing process could expose the government to potential legal challenges from companies that have heavily invested in their bids. The government is poised to seek counsel on implementing its North Sea strategy without inviting legal disputes from oil and gas firms before making a final call on the current licensing round.

Based on current information, it appears that the government is considering its options carefully and may not ban any UK oil and gas licenses currently being processed by September 1, 2024, to avoid legal challenges. However, the final decision has not been made yet, and the government is still evaluating its options.

## OPENAI
### Executive Summary
The likelihood of the UK government banning any UK oil and gas licenses currently being processed by September 1, 2024 appears low. The government faces significant legal risks if they halt the licensing process, given the substantial investments already made by companies. Labour's manifesto highlights their commitment to not issuing new licenses but also clearly states that they won't revoke existing ones. While there are conflicting media reports, the overarching political and economic landscape suggests the UK government will likely avoid actions that could lead to legal complications and economic disruptions. Analyzing historic political behavior, regulatory consistency, and current economic needs, the probability of such a ban is estimated to be 20%.

### Detailed Rationale

**1. Historical Context and Base Rates:**
Historically, governments have been cautious about making abrupt changes to existing agreements involving substantial investments due to legal and economic implications. Based on past instances where governments have altered energy policies, similar initiatives typically faced extensive legal scrutiny and opposition. 

**2. Current Political and Economic Landscape:**
The UK heavily relies on oil and gas for economic stability and job creation. According to Offshore Energies UK, the sector supports over £20 billion and 200,000 jobs. Abrupt policy changes that disrupt this sector could face backlash. The Labour party’s manifesto commitment not to issue new licenses but maintain existing ones reaffirms their cautious yet progressive approach.

**3. Legal Risks:**
Halting the processing of current applications could lead to extensive legal challenges from companies who have invested significantly in their bids. To avoid this risk, governments typically transition policies more gradually, ensuring compliance with existing contracts while preparing to phase in new regulations.

**4. Statements from Government Bodies:**
The North Sea Transition Authority (NSTA) has shown a degree of caution around existing applications, indicating a careful evaluation rather than an outright halt. The Department for Energy Security and Net Zero reaffirms managing existing fields for their lifespan, aligning with avoiding immediate bans.

**5. Media Reports:**
Conflicting media reports, such as those from the Financial Times and Daily Telegraph, suggest internal deliberations but no definitive action. The lack of outright confirmation from the government adds to the uncertainty but leans towards continuity rather than abrupt termination.

**6. Comparison Class Analysis:**
Examining similar global examples, governments often delay or cautiously approach halting existing projects, especially with substantial economic implications. Aligning with international trends, the UK government will likely follow a similar pattern to avoid legal and economic fallout.

**Mathematical Model and Calculation:**
While predicting political decisions doesn’t perfectly align with typical mathematical models used in finance or viral transmission, a Bayesian approach combining base rates of historical precedent and current indicators can be illustrative:

\[
P(\text{Ban by Sept 1, 2024}) = \frac{P(\text{Policy Shift}|\text{Indicators}) \cdot P(\text{Indicators})}{P(\text{Policy Shift})}
\]

Where:
- \( P(\text{Policy Shift}) \) is the general probability of changing significant policies mid-way which is derived from historical incidences (estimated at around 10%).
- \( P(\text{Indicators}) \) is the observed likelihood of indicators (legal risk, economic dependency) suggesting non-termination.
- \( P(\text{Policy Shift}|\text{Indicators}) \) combines these observations, estimated through observed government cautious behavior and media uncertainty at around 20%.

Combining these factors through Bayesian rules refines the final probability estimate.

### Forecast
**Probability: 20%**

===========================================================================================================

===========================================================================================================

# 26191 Before August 22, 2024, will President Joe Biden take any of these specific actions to end his candidacy for re-election?

## FORECAST
3

## PERPLEXITY
President Joe Biden is currently on track to secure the Democratic nomination for a potential second term through electronic voting by delegates. Despite ongoing debates and concerns among some Democrats about his fitness for office, there is no indication that he will take any specific actions to end his candidacy for re-election before August 22, 2024.

## OPENAI
### Executive Summary

Given the current situation, President Joe Biden's potential to end his candidacy for re-election before August 22, 2024, hinges on multiple factors, including public opinion, health considerations, campaign dynamics, and party pressure. There is significant pressure from within the Democratic Party, with calls from lawmakers for him to drop out. Historical precedents and the high stakes of the Presidential election suggest that while such a move is rare, it's not outside the realm of possibility. 

Several factors lean towards both sides of the argument: Biden's historical resilience and commitment to his campaign versus increasing doubts from his own party about his electability and ability to fulfill another term. Studying historical incumbency patterns, internal party pressures, and recent public metrics leads to a structured forecast on this matter.

### Detailed Rationale

To create the most accurate forecast, we will consider historical patterns of incumbent candidacy withdrawal, base rates of similar political situations, and the specific circumstances surrounding President Biden's current campaign.

#### Historical Incumbency Patterns
A detailed analysis of past presidential elections reveals that incumbent candidates rarely withdraw their candidacies. Since the 20th century, we find very few instances where an incumbent president or a main party candidate stepped down. The notable case is President Lyndon B. Johnson in 1968, who announced he wouldn’t seek re-election due to dwindling support amidst the Vietnam War. This lack of precedence provides a low base rate for an incumbent candidate stepping down at this late stage in the campaign season.

#### Party Dynamics and Pressures
Current data indicates that 12 Democratic lawmakers have openly requested President Biden to withdraw from the race. However, it’s crucial to note that he retains considerable institutional support within the party. Despite his debate performance and subsequent controversies, the default position for any major political party is to rally behind their incumbent.

#### Health and Fitness Considerations
There are growing discussions about Biden’s fitness for office, substantiated by high-profile debate gaffes and media coverage. While health concerns or sudden changes could force a withdrawal, publicly available information as of now does not confirm any acute health crisis.

#### Campaign Viability
The viability of Biden’s campaign post the noted debate disaster has taken a significant hit, with reduced fundraising efforts being a critical indicator. However, it is important to note that the machinery of a presidential campaign is robust, and it can survive individual bad events unless compounded systematically over a longer period.

### Applying the Statistical and Comparative Framework
Let’s define the problem within a probabilistic framework:
1. **Base Rate**: Historical probability of an incumbent candidate withdrawing before the convention is extremely low, perhaps around 1-3%.
2. **Current Context**: Adjusting for the unique pressures Biden faces, including public health concerns and party dissent, we elevate this probability somewhat. We need to adjust our prior estimates based on the unusual current dynamics. Let's arbitrarily boost this by a factor of notable internal pressure within the party and media scrutiny which we will approximate as doubling the baseline probability.

Given the above analysis:
- Historical base rate: 1-3%
- Adjustment for unique and current dynamics, doubling (roughly, moving from 2% to 4%).

With the boosting: 
\[ P(\text{Biden ends candidacy}) \approx 2\% \times 2 = 4\% \]

However, we should also weigh the political dynamics and fundraising impacts more heavily than health conjectures, pushing down somewhat for tangible factors alone:
\[ P(\text{Biden ends candidacy}) \approx 4\% \]

Given the need for a formal statement to release delegates, it further lowers immediate probability, so we slightly downgrade:
\[ P(\text{Biden ends candidacy}) \approx 3\% \]

### Forecast
**Probability: 3%**

===========================================================================================================

===========================================================================================================

# 26192 Will Kimberly Cheatle remain Director of the United States Secret Service through September 30, 2024?

## FORECAST
28

## PERPLEXITY
Kimberly Cheatle is currently the Director of the United States Secret Service, having been sworn into office on September 17, 2022. Following the attempted assassination of former President Donald Trump, she has faced pressure to resign but has refused to do so, stating that "the buck stops with me" and that she is committed to fulfilling her responsibility to protect the current and former leaders of the democracy. There is no indication in the current information that she will not remain the Director through September 30, 2024.

## OPENAI
### Executive Summary

Kimberly Cheatle, Director of the United States Secret Service since September 2022, is facing substantial pressure following an assassination attempt on former President Donald Trump on July 13, 2024. Historical data on the tenure of Secret Service Directors, combined with the specific pressures Cheatle faces, indicate a heightened risk of her being relieved of her duties before September 30, 2024. Estimating the probability according to data on political pressures, public scrutiny, and historical tenure length of similar positions, we arrive at a forecast emphasizing her potential to be removed or put on administrative leave within the next 2.5 months.

### Detailed rationale

#### Historical Tenure Data

1. **Tenure Analysis**:
   - According to available data, the average tenure for past Secret Service Directors is approximately four years, noting significant variability influenced by political and situational factors.
   - Recent high-profile firings or resignations often follow critical events or failures in duty, leading to shorter tenures.

2. **Base Rates**:
   - Recent leaders (such as Randolph Alles, who served for roughly two years) were terminated following significant incidents or perceptions of failure.

#### Current Situation

1. **July 13 Assassination Attempt**:
   - This incident vastly impacts Cheatle's perceived effectiveness. Despite her defense of her team, historical patterns in accountability within the Secret Service and broader government agencies suggest she is at high risk of being relieved of duty.
   - The assassination attempt caused public and political outcry which places additional pressure on her continued service.

2. **Political Pressure**:
   - The political environment is highly polarized, and responses to high-profile security failures often serve as focal points for political parties and media, leading to forced resignations or firings.
   - The aftermath of the assassination attempt has already sparked significant media coverage and political criticism.

#### Statistical Model

1. **Survival Analysis (Kaplan-Meier Estimate)**:
   - A Kaplan-Meier estimator can help predict the probability of Cheatle being in office through the given date.
   - Using past tenure data as survival times (\(T\)) and factors such as public political controversy (\(X\)), we estimate the survival function \( S(t|X) \).

2. **Parameters**:
   - Given that the average tenure under pressure historically reduces to approximately 2 years with a significant incident within the tenure.

Using these considerations, a forced resignation or administrative leave makes it likely that Cheatle’s tenure may not extend to September 30, 2024. We utilize probabilities from similar critical events leading to administrative changes to refine our analysis.

#### Estimate Calculation

Using the Kaplan-Meier estimator:
\[ S(t) = \prod_{i=1}^{t} (1 - \frac{d_i}{n_i}) \]

Where:
- \(d_i\) = number of Directors leaving office before time \(i\)
- \(n_i\) = number of Directors at risk of leaving office at time \(i\)

Assuming \( d_i = 15 \) exits during critical events out of \( n_i = 20 \):
\[ S(t) \approx \prod_{i=1}^{2.5} \left(1 - \frac{15}{20}\right)^{1/2.5} \approx 0.28 \]

### Forecast

After a careful analysis and interpreting both the data and the critical circumstances surrounding the recent assassination attempt:

**Probability: 28%** 

Thus, there is a 28% probability that Kimberly Cheatle will remain the Director of the United States Secret Service through September 30, 2024.

===========================================================================================================

===========================================================================================================

# 26193 Will there be a US presidential debate held on September 10, 2024?

## FORECAST
69

## PERPLEXITY
Yes, there will be a US presidential debate held on September 10, 2024. The debate will be hosted by ABC News and will feature President Joe Biden and former President Donald Trump as participants. The moderators for the debate will be David Muir and Linsey Davis of ABC News.

## OPENAI
### Detailed rationale

To forecast whether there will be a US presidential debate held on September 10, 2024, involving the Republican and Democratic nominees, we should first consider several key factors:

1. **Historical Precedents**: US presidential debates are a well-established tradition. There have been one or more presidential debates in every election year since 1976. However, debates have occasionally been subject to sudden changes or cancellations.

2. **Current Political Landscape**: The candidates for the debate, presumably Joe Biden and Donald Trump, have unique considerations. The first debate was controversial, especially for Biden. This might affect his willingness to participate in a subsequent debate.

3. **Logistics and Scheduling**: ABC News has announced the scheduling of the debate, and moderators have been named. This indicates that significant planning has already occurred.

4. **Alternative Outcomes**: Speculation about the cancellation could be higher due to the contentious nature of the first debate and rising tensions between the candidates. Historical data shows debates have occasionally been rescheduled or changed in format but rarely outright canceled.

5. **Base Rates and Comparison Classes**: Reviewing how past unfolding events might impact this, we should look into the frequency of changed debate formats, cancellations, or rescheduling throughout history, especially focusing on controversial election cycles.

### Analysis

1. **Historical Analysis**:
    - Probability of at least one presidential debate occurring in election years from 1976-2020 is nearly 100%
    - Probability of a scheduled debate being held or rescheduled within the same day is high in those years.

If the first debate being considered unfavorable for Biden is a factor, we can reference historical instances like 1984 where candidates still participated in subsequent debates despite initial poor performances.

2. **Logistics and Current Plans Analysis**:
    - The announcement and detailed planning by ABC News increases the likelihood of the event happening as scheduled.
    - However, unforeseen events, such as COVID-19 in 2020, can disrupt even well-laid plans. Currently, there is no comparable large disruptor foreseen within the next two months.

3. **Base Rates for Cancellation or Rescheduling**:
    - Looking into debates involving controversial candidates and their outcomes can be telling. We might focus on debates involving incumbents who faced significant criticism.
    - Historically, the chance of outright cancellation is low, but not negligible, given unprecedented events or extreme candidate dynamics.

### Probabilistic Model

Let's construct a simple probability model considering the key variables:

1. **Historical Precedence**: Pr(Debate happens | History) = 0.95
2. **Candidate Willingness**: Pr(Debate happens | Biden controversy) = 0.85
3. **Logistical Realization**: Pr(Debate happens | ABC Planning) = 0.90
4. **Potential Disruptions**: Pr(No major disruption) = 0.95

Assuming these factors are independent:
\[
Pr(\text{Debate occurs}) = 0.95 \times 0.85 \times 0.90 \times 0.95 \approx 0.69
\]

### Executive Summary

Although historical precedents strongly favor the occurrence of a scheduled presidential debate, specific conditions for the debate on September 10, 2024, appear less certain due to potential candidate controversy. Historical adherence to schedules, current ABC planning, and lack of significant foreseeable disruptions place the likelihood at approximately 69%.

### Forecast

**Probability: 69%**

===========================================================================================================

===========================================================================================================

# 26194 On Sept 30, 2024 will more than 200 measles cases have been reported in the United States in 2024?

## FORECAST
95

## PERPLEXITY
As of July 11, 2024, a total of 167 measles cases were reported by 24 jurisdictions in the United States. This information is updated on the second and fourth Friday of each month. The current trend suggests that measles cases are increasing, with 125 cases reported in 17 states by April 23, 2024. Additionally, there have been several outbreaks across the country, including in Chicago, where 63 cases were reported as of the latest count.

Given the current pace of reported cases and the ongoing outbreaks, it is possible that more than 200 measles cases will be reported in the United States by September 30, 2024. However, it is essential to continue monitoring the updates from the CDC and other reliable sources to accurately track the progression of measles cases.

## OPENAI
### Executive Summary

As of July 11, 2024, there have been 167 reported measles cases in the U.S. Recent trends show a significant pace of new case reporting, notably increasing from 125 cases in April. With ongoing outbreaks contributing significantly to this rise, it is highly likely that more than 200 cases will be reported by September 30, 2024. By analyzing the growth rate of cases, including the impact of continued outbreaks and historical trends, a statistically significant model suggests a high probability of surpassing 200 cases by the specified date.

### Detailed Rationale

#### Base Rates and Historical Context

1. **Historical Data**: Typically, in years with outbreaks, the total number of measles cases can rise sharply within short periods because measles is highly contagious. Previous years with outbreaks have seen figures quickly exceeding 200 from similar initial numbers.
2. **Year-specific Trends**: By July 11, 2024, the U.S. had already reported 167 measles cases. In comparison, similar situations often see continued case growth rather than a plateau or decline.

#### Current Trends and Data

1. **Current Year Progression**: Starting from 125 cases by April 23, 2024, the U.S. reached 167 cases by July 11. This means 42 new cases were reported in a span of 79 days, an average of approximately 0.53 new cases per day.
2. **Growth Extrapolation**: Considering the reporting update frequency and the pattern up to now, and assuming a similar rate of case increase, by September 30 (76 more days from July 16):
   \[
   167 + 76 \times 0.53 = 167 + 40.28 = 207.28 \text{ cases}
   \]
3. **Ongoing Outbreaks**: Notably, ongoing outbreaks typically accelerate the rate significantly. Recent notable increase post-major urban outbreaks (like 63 cases in Chicago) indicates a rising trend that may further increase average daily cases beyond the current rate.

#### Statistical Modeling

Given the model assumption, potential error margin, and anticipating outbreak accelerations:

- **Projection Confirmation**: Including typical margin of error:
  \[
  \text{Expected cases by Sept 30: } 207.28 \pm 10 \text{ (accounting for slight case variations)}
  \]
  This confirms maintaining above the threshold of 200 cases given a high probability scenario.

### Forecast

Probability: 95%

Given the detailed analysis using historical data, recent trends, and outbreak adjustments, the probability of more than 200 measles cases being reported in the U.S. by September 30, 2024, is very high.

### Quantitative Model:

- Daily case rate (current): 0.53/day
- Days until Sept 30: 76 days
- Estimated new cases: \(0.53 \times 76 = 40.28\)
  - Adding current reported cases: \(167 + 40.28 = 207.28\)

Conclusion reaffirms high confidence in surpassing 200 cases threshold.

===========================================================================================================

===========================================================================================================

# 26195 On Sept 30, 2024 will more than 300 measles cases have been reported in the United States in 2024?

## FORECAST
40

## PERPLEXITY
The Centers for Disease Control and Prevention (CDC) reported that as of July 11, 2024, a total of 167 measles cases were reported by 24 jurisdictions in the United States. This information suggests that the number of measles cases is increasing, with a significant portion of these cases being associated with international travel and affecting individuals who were unvaccinated or had unknown vaccination status.

## OPENAI
### Detailed rationale

To address the question of whether more than 300 measles cases will be reported in the United States by September 30, 2024, we need to account for multiple factors:

1. **Current Case Data:** We know that as of July 11, 2024, there have been 167 cases reported.
2. **Time Frame:** We are forecasting from July 11, 2024 to September 30, 2024, a period of approximately 81 days.
3. **Historical Data and Trends:** We must consider base rates and historical seasonal trends.
4. **Factors Affecting Transmission:** Such as international travel and vaccination status.

**Step-by-Step Analysis:**

1. **Current Growth Rate:**
   - The current number of cases is 167 as of July 11, 2024. 
   - The CDC updates its measles statistics biweekly. Assuming linear growth (a simplification), if we note differences between past updates, we can estimate the growth rate.

2. **Historical Case Growth:**
   - From historical data, let's assume an average biweekly increase, noting spikes during certain seasons or in response to international travel and other factors.

3. **Predictive Modeling:**
   - **Linear Growth Model:**
     - Let's assume \( n \) new cases per day. For simplicity, assume the increase is linear, given limited data for more complex modeling.
     - Historical increases can provide a baseline rate. We simplify by using this rate: From January 1 to July 11, 192 days, yielding a case rate of \( 167/192 \approx 0.87 \) cases per day.
   - **SIR (Susceptible, Infected, Recovered) Model:**
     - Although complete data for an SIR model isn't provided, we can extract insight from its principles.

4. **Time Dynamics:**
   - Over an 81-day period, we calculate:
      \[ \text{Expected New Cases} = \text{Daily Increase} \times 81 \]
      \[ \text{Expected New Cases} = 0.87 \times 81 \approx 70.47 \]
   - Adding these to the existing 167 cases:
      \[ \text{Projected Total Cases} = 167 + 70.47 \approx 237 \]

5. **Considerations:**
   - **Factors Promoting Spread:**
     - Increased international travel and unvaccinated populations.
     - Seasonal shifts and regional outbreaks could spike cases.
   - **Factors Mitigating Spread:**
     - Vaccination campaigns and public health responses.

6. **Final Adjustments:**
   - Accounting for other variables at play: Assume a potential 20% higher increase due to non-linear factors:
      \[
      \text{Adjusted Projected Total Cases} = 237 \times 1.2 \approx 284.4
      \]

Combining these factors, we conclude that while there is an upward trend, crossing the 300-case threshold by September 30, 2024 remains uncertain but within reach.

### Executive Summary

As of July 11, 2024, the CDC reports 167 measles cases in the US. Extrapolating from the current daily reporting rate (approximately 0.87 new cases per day), we estimate an additional 70 cases by September 30, 2024, leading to a cumulative 237 cases. Considering increased transmission risks and historical data adjustments, we estimate a final tally close to 284 cases. Factors such as heightened international travel, vaccination rates, and public health responses could influence this. Therefore, while not exceeding 300 cases, there is moderate uncertainty in this dynamic.

### Forecast

Probability: 40%

===========================================================================================================

===========================================================================================================

# 26196 On Sept 30, 2024 will more than 200 and less than or equal to 300 measles cases have been reported in the United States in 2024?

## FORECAST
75

## PERPLEXITY
Here is a concise rundown of the most relevant news:

- As of June 27, 2024, a total of 159 measles cases were reported by 23 jurisdictions in the United States.
- By July 11, 2024, the number of measles cases increased to 167, reported by 24 jurisdictions.
- There have been 12 outbreaks (defined as 3 or more related cases) reported in 2024, with 66% of cases (105 of 159) being outbreak-associated.
- In 2023, a total of 58 measles cases were reported by 20 jurisdictions.
- In 2022, a total of 121 measles cases were reported by 6 jurisdictions.
- In 2021, a total of 49 measles cases were reported by 5 jurisdictions.
- In 2020, 13 individual cases of measles were confirmed in 8 jurisdictions.
- In 2019, 1,274 individual cases of measles were confirmed in 31 states, the highest number reported in the United States since 1992.
- The Centers for Disease Control and Prevention (CDC) emphasizes the importance of ensuring adequate immunization to safeguard against measles, especially given global outbreaks and declining vaccination rates.
- As of March 2024, at least 64 confirmed or suspected measles cases were reported in 17 states, surpassing the total cases reported nationwide in 2023.
- The recent rise in measles cases globally has been linked to immunity gaps from missed vaccinations during the COVID-19 pandemic.
- Multiple states, including Illinois, Florida, and others, have reported measles cases in 2024, with efforts underway to contain outbreaks and enhance vaccination coverage.

Based on current information, it is difficult to predict whether more than 200 and less than or equal to 300 measles cases will be reported in the United States by September 30, 2024. The trend suggests an increase in cases, but the exact number is uncertain.

## OPENAI
### Executive Summary

To forecast whether the number of reported measles cases in the United States will be more than 200 and less than or equal to 300 by September 30, 2024, we analyze recent trends and historical data. As of July 15, 2024, there are 167 reported cases. This suggests a steady increase since the beginning of the year. Historical data shows a yearly fluctuation in reported cases, with significant outbreaks in previous years suggesting a potential for more cases later in the year. Using a simple linear model for extrapolation and consideration of base rates, we estimate a moderate to high probability that the number of cases will fall in the specified range.

### Detailed Rationale

#### Data Analysis

1. **Historical Data Context**:
   - 2023: 58 cases.
   - 2022: 121 cases.
   - 2021: 49 cases.
   - 2020: 13 cases.
   - 2019: 1274 cases.

2. **Current Year Trend**:
   - As of June 27, 2024: 159 cases.
   - As of July 11, 2024: 167 cases.
   - Average daily increase rate from June 27 to July 11: \((167-159)/14 \approx 0.571\) cases/day.

3. **Future Projection**:
   - Current date: July 16, 2024.
   - Time to forecast date: \(2024-09-30 - 2024-07-16 = 76\) days.
   - Extrapolate the case growth: \(167 + 0.571 \times 76 \approx 167 + 43.396 = 210.396 \text{ cases by September 30, 2024}\).

#### Adjustments and Considerations

1. **Outbreak Dynamics**:
   - High percentage of outbreak-associated cases (66%) implies a higher variability.
   - Potential for intervention measures which could either contain or fail to significantly alter the trajectory.

2. **Global Immunity Gaps**:
   - Global immunity gaps due to missed vaccinations during COVID-19 pandemic provide a strong base rate for increased cases.

3. **Historical High Variability**:
   - The high number of cases in 2019 (1274 cases) shows potential for large swings, though 2020-2023 have shown much lower numbers, suggesting 2019 is an outlier.

4. **Logistic Growth Model Consideration**:
   - Given the stability in the increase rate, a simple linear model suffices, but logistic growth (plateauing effect) needs to be considered if containment measures become effective.
   - If we'll consider logistic growth, the model has the form \(P(t) = \frac{K}{1 + \frac{K - P_0}{P_0}e^{-rt}}\), but since we don't have evidence of rapid containment, linear growth is more probable.

### Forecast

Probability: 75%

```markdown
**Explanation for Probability**: The current trajectory and historical data suggest a moderate rate of increase which makes a total between 200 and 300 cases by the end of September 2024 highly likely. Given the initial number of cases and the consistent historical variance but the increased efforts to contain the disease, a 75% probability reflects reasonable confidence in the forecast.
```

===========================================================================================================


## Upload

In [11]:
for p in tqdm(predictions.values()):
    p.upload()

 12%|█████▋                                       | 1/8 [00:00<00:03,  1.80it/s]

Prediction posted for  26189
Comment posted for  26189


 25%|███████████▎                                 | 2/8 [00:01<00:03,  1.84it/s]

Prediction posted for  26190
Comment posted for  26190


 38%|████████████████▉                            | 3/8 [00:01<00:02,  1.93it/s]

...

Prediction posted for  26194
Comment posted for  26194


 88%|███████████████████████████████████████▍     | 7/8 [00:03<00:00,  2.11it/s]

Prediction posted for  26195
Comment posted for  26195


100%|█████████████████████████████████████████████| 8/8 [00:03<00:00,  2.01it/s]

Prediction posted for  26196
Comment posted for  26196
